## Modélisation de la survie d'un individu dans un milieu hostile.

In [385]:
import numpy as np

In [499]:
def construction_abri(cap , source_inf , vegr , veg , s , jourres , abri):
    """float*bool*int*float*int*bool => bool*int*bool*bool"""
    #sauve:bool
    sauve = False
    #rand:float
    if jourres != 0:
        jourres = jourres - 1
    else:
        abri = True
    if jourres == -11 or vegr == 0:
        jourres = ((45/(cap*0.5)//10 * 1/(veg/5)))/1.
        abri = False
        source_inf = False
        rand = np.random.random_sample()
        if rand < s:
            sauve = True
    jourres = jourres//1

    return jourres , abri , sauve , source_inf

In [387]:
construction_abri(0.1,False,1,20,0.1,-10,False)

(22.0, False, True, False)

In [388]:
def trouver_eau(p_eau , capadap , source_inf , hydrat):
    """float*float*bool*float => bool*float"""
    
    if source_inf == 1:
        hydrat = 1
    else:
        rand = np.random.random_sample()
        if rand <= 0.4*p_eau*(2*capadap):
            
            hydrat = hydrat + 0.2
        elif rand <= p_eau*(4*capadap) :
            
            hydrat = hydrat + 0.5
            rand=np.random.random_sample()
            if rand > 0.4:
                b=3
                source_inf = True
    if hydrat > 1:
        hydrat = 1
    return source_inf , hydrat

In [389]:
 def inter_veget(veg,alimentation,capadap,vegre):
    """int*float*float=>int*float"""
    #p_vg:float
    p_vg=0.6
    #trouve:bool
    trouve=True
    #vegr:int
    vegr=vegre
    if vegr == 0:
        vegr = veg
    while vegr>0 and trouve==True:
        rand=np.random.random_sample()
        if rand * capadap < p_vg:
            alimentation = alimentation + (0.08 * capadap)
            vegr = vegr - 1
            p_vg = p_vg-0.1
        else:
            trouve=False
    if alimentation > 1:
        alimentation = 1
    return alimentation , vegr

In [401]:

def inter_anim(p_a, p_ap, force, capadap, alimentation, sante):
    """
    float ^6 -> float ^2
    hyp : 
    """
    random1=np.random.random_sample()
    if random1 <= p_a : 
        random2 = np.random.random_sample()
        if random2 <= p_ap :
            alimentation,sante = inter_pred(force, capadap, alimentation, sante)
        else :
            alimentation = alimentation + 0.15
    if alimentation > 1:
        alimentation = 1.0
        
    return alimentation, sante

In [399]:
def inter_pred(force, cap, alimentation, sante):
    """
    float * float * float * float -> float * float 
    hyp : random2 <= p_a
    rend le niveau d alimentation et de sante apres une rencontre avec un animal dangereux 
    """

    random4=np.random.random_sample()
    combat = random4*(force+0.5)*(cap+0.5)
    if combat <= 0.1 :
        sante = 0 
    elif combat <=0.4 :
        sante = sante - (1-force)*(1-cap)*0.5
    elif combat <= 0.7 : 
        sante =sante - (1-force)*(1-cap)*0.4
        alimentation = alimentation + 0.15 
    else :
        alimentation = alimentation + 0.15
    return alimentation, sante

In [540]:
def sante_fct_hydr_alim(alimentation,hydratation,sante):
    """float³=>float
    force en fonction de l'alimentation et hydratation"""
    if alimentation >= 0.5 and hydratation >=0.5:
        if sante<0.3:
              sante = 0.3
        else:
              sante = sante + 0.1
    elif alimentation<0.5 :
        if hydratation<0.5:
            sante = sante - ((0.5-alimentation)*0.5 + (0.5-hydratation)*0.8)
        else:
            sante = sante - (0.5-alimentation)*0.4
    else:
        sante = sante - (0.5-hydratation)*0.84
    if sante > 1.0 :
        sante = 1.0
    
    hydratation=hydratation-0.3
    alimentation=alimentation-0.2
    if hydratation <=0 :
        sante =0
    return sante,alimentation,hydratation

In [463]:
def secouru(p_s, j,p_s_ini,p_s_10):
    """
    float * int -> bool 
    hyp :
    retourne si l agent est secouru par une equipe de secours
    """


    #secouru : bool
    secouru = False

    if j <= 10 :
        p_s = p_s + p_s_ini*0.2
    elif j <= 25 :
        p_s_10 = p_s
    else :
        p_s = p_s - p_s_10*0.2
    
    random5=np.random.random_sample()
    secouru = random5 <= p_s

    return secouru , p_s


In [464]:
secouru(0.008666657999999999,0,0.00666666,0)

(False, 0.009999989999999999)

In [465]:
#dictionnaire_agent:dict[str:tuple[float,float,float]]
dictionnaire_agent={}
dictionnaire_agent["professionnel"]=(0.8,0.9)
dictionnaire_agent["bureaucrate"]=(0.2,0.3)
dictionnaire_agent["moyen"]=(0.5,0.5)

#dictionnaire_biome:dict[str:tuple[float,float,int,float,float,float]]
dictionnaire_biome={}
dictionnaire_biome["Amazonie"]=(0.65,0.5,33,0.7,0.03571428571,0.007142857142)
dictionnaire_biome["Alaska"]=(0.38,0.25,18,0.6,0.00043185564,0.00086371128)
dictionnaire_biome["France"]=(0.3,0.05,17,0.6,0.03,0.06)
dictionnaire_biome["Sahara"]=(0.25,0.5,1,0.1,0.00333333,0.00666666)

In [541]:
def simulation_survie(agent,biome):
    """str*str*dict[str:tuple[float,float]]*dict[str:tuple[float,float,int,float,float,float]]"""
    
    #c : cause de la fin
    c = ""
    #j : compteur jour
    j = 0
    #jour_r:float
    jour_r=-10
    #abri:bool
    abri=False
    #sauve:bool
    sauve=False
    #sourceinf:bool
    source_inf=False
    #sante:float
    sante=1.0
    #hydrat:float
    hydrat=1.0
    #alimentation:float
    alimentation=1.0
    
    force , capadap = dictionnaire_agent[agent]
    p_a , p_ap , veg , p_e , p_v , p_s = dictionnaire_biome[biome]
    vegr=veg
    p_s_ini = p_s
    p_s_10=p_s
    while sante > 0 and sauve != True and j!=60:
        jour_r , abri , sauve , source_inf = construction_abri(capadap , source_inf , vegr , veg , p_v , jour_r , abri)
        sourceinf , hydrat = trouver_eau(p_e , capadap , source_inf , hydrat)
        alimentation , sante = inter_anim(p_a, p_ap, force, capadap, alimentation, sante)
        alimentation , vegr = inter_veget(veg,alimentation,capadap,vegr)
        sante,alimentation,hydrat = sante_fct_hydr_alim(alimentation, hydrat, sante)
        sauve , p_s = secouru (p_s,j,p_s_ini,p_s_10)
        print(j,sante,alimentation,hydrat)
        j = j + 1
        if j==10:
            p_s_10 = p_s


    return j, c,sante,force,capadap,sauve


In [543]:
simulation_survie("bureaucrate","France")

0 1.0 0.8 0.7
1 1.0 0.8 0.7
2 1.0 0.7200000000000002 0.39999999999999997
3 1.0 0.6940000000000002 0.5999999999999999
4 1.0 0.5900000000000003 0.2999999999999999


(5, '', 1.0, 0.2, 0.3, True)

In [479]:
def test(a,b,c):
    while a!=0:
        print(simulation_survie(b,c))
        a=a-1
    return simulation_survie(b,c)

In [482]:
test(10,"professionnel","Sahara")

(60, '', 1.0, 0.8, 0.9, False)
(60, '', 1.0, 0.8, 0.9, False)
(5, '', 0.5, 0.8, 0.9, True)
(60, '', 1.0, 0.8, 0.9, False)
(4, '', 1.0, 0.8, 0.9, True)
(19, '', 0.4, 0.8, 0.9, True)
(60, '', 0.7999999999999999, 0.8, 0.9, False)
(23, '', 1.0, 0.8, 0.9, True)
(14, '', 1.0, 0.8, 0.9, True)
(60, '', 1.0, 0.8, 0.9, False)


(60, '', 1.0, 0.8, 0.9, False)